# Short and Long Term Energy Model - Isle of Eigg

##### Created on: 25.12.2023
##### Last updated on: 17.01.2024
##### File Revision: 01

##### Developed by: Taha Ahmed Siddiqui (tahaahmedsiddiqui@outlook.com) and Hareem Taha (hareem.nadeem10@gmail.com)
##### GitHub Repository: https://github.com/TahaAhmedSiddiqui/Energy-Modelling---Isle-of-Eigg

### 1. Base Optimization Model

##### (Define the existing configuration of the model here)

##### Enter your code here and before 2. for Base Optimization Model

In [196]:
#Import relevant packages here

import pyomo.environ as pyo
import pandas as pd
import matplotlib.pyplot as plt

In [197]:
#Initiate the model

#Naming the model 'EM' -- stands for Energy Model
EM = pyo.AbstractModel()

#### Sets

In [198]:
#Call the data from input csv's and generate sets
#The model iterates over a set of technologies, a set of storages, and a set of fuels that connects the two and serves the demand

#Reading the csv's
technology = pd.read_csv('../1. Input/technologies.csv')
fuel = pd.read_csv('../1. Input/fuels.csv')
storage = pd.read_csv('../1. Input/storages.csv')

#Creating sets
EM.Technology = pyo.Set(initialize = technology['technology'])
EM.Fuel = pyo.Set(initialize = fuel['fuel'])
EM.Storage = pyo.Set(initialize = storage['storage'])
EM.Hour = pyo.RangeSet(1,120,1)

#### Parameters

In [199]:
#Call the data from input csv's and generate parameters
#The model has the installed capacities of technologies and storages, their conversion ratios and efficiencies, and a demand profile

#Reading the csv's
readin_variablecost = pd.read_csv('../1. Input/variablecost.csv', index_col=[0], sep=',').squeeze('columns')
readin_inputratio = pd.read_csv('../1. Input/inputratio.csv', index_col=[0,1], sep=',').squeeze('columns')
readin_outputratio = pd.read_csv('../1. Input/outputratio.csv', index_col=[0,1], sep=',').squeeze('columns')
readin_demand = pd.read_csv('../1. Input/demand.csv', index_col=[0], sep=',').squeeze('columns')
readin_emissionratio = pd.read_csv('../1. Input/emissionratio.csv', index_col=[0], sep=',').squeeze('columns')
readin_maxcapacity = pd.read_csv('../1. Input/maxcapacity.csv', index_col=[0], sep=',').squeeze('columns')
readin_demandprofile = pd.read_csv('../1. Input/demand_timeseries.csv', index_col=[0,1], sep=',').squeeze('columns')
readin_capacityfactor = pd.read_csv('../1. Input/capacity_factors_2018.csv', index_col=[0,1], sep=',').squeeze('columns')
readin_tagdispatchabletechnology = pd.read_csv('../1. Input/tagdispatchabletechnology.csv', index_col=[0], sep=',').squeeze('columns')
readin_e2pratio = pd.read_csv('../1. Input/e2pratio.csv', index_col=[0], sep=',').squeeze('columns')
readin_storagechargeefficiency = pd.read_csv('../1. Input/storagechargeefficiency.csv', index_col=[0,1], sep=',').squeeze('columns')
readin_storagedischargeefficiency = pd.read_csv('../1. Input/storagedischargeefficiency.csv', index_col=[0,1], sep=',').squeeze('columns')
readin_investmentcost = pd.read_csv('../1. Input/investmentcost.csv', index_col=[0], sep=',').squeeze('columns')
readin_investmentcoststorage = pd.read_csv('../1. Input//investmentcoststorage.csv', index_col=[0], sep=',').squeeze('columns')
readin_residualcapacity = pd.read_csv('../1. Input/residualcapacity.csv', index_col=[0], sep=',').squeeze('columns')

#Creating parameters
EM.VariableCost = pyo.Param(EM.Technology, default=0, initialize=readin_variablecost)
EM.OutputRatio = pyo.Param(EM.Technology, EM.Fuel, default=0, initialize=readin_outputratio)
EM.InputRatio = pyo.Param(EM.Technology, EM.Fuel, default=0 , initialize=readin_inputratio)
EM.Demand = pyo.Param(EM.Fuel, default =0, initialize=readin_demand)
EM.EmissionRatio = pyo.Param(EM.Technology, default=0, initialize=readin_emissionratio)
EM.MaxCapacity = pyo.Param(EM.Technology, default=999, initialize=readin_maxcapacity)
EM.DemandProfile = pyo.Param(EM.Fuel, EM.Hour, default=1/len(EM.Hour.data()), initialize=readin_demandprofile)
EM.CapacityFactor = pyo.Param(EM.Technology, EM.Hour, default=0, initialize=readin_capacityfactor, mutable=True)
EM.TagDispatchableTechnology = pyo.Param(EM.Technology, default=1, initialize=readin_tagdispatchabletechnology)
EM.E2PRatio = pyo.Param(EM.Storage, default=0, initialize=readin_e2pratio)
EM.StorageChargeEfficiency = pyo.Param(EM.Storage, EM.Fuel, default=0, initialize=readin_storagechargeefficiency)
EM.StorageDisChargeEfficiency = pyo.Param(EM.Storage, EM.Fuel, default=0, initialize=readin_storagedischargeefficiency)
EM.EmissionLimit = pyo.Param(initialize=999999)
EM.CapitalCost = pyo.Param(EM.Technology, default=0, initialize=readin_investmentcost)
EM.InvestmentCostStorage = pyo.Param(EM.Storage, default=0, initialize=readin_investmentcoststorage)
EM.ResidualCapacity = pyo.Param(EM.Technology,default=0,initialize=readin_residualcapacity)

In [200]:
# this block of code sets the CapacityFactor of technologies that are dispatchable to be always 1
# first, we need to construct our sets in order to be able to loop over them
EM.Technology.construct()
EM.TagDispatchableTechnology.construct()
EM.CapacityFactor.construct()
# then, we can loop over t and h and set the CapacityFactor to always be 1 when TagDispatchableTechnology is non-zero
for t in EM.Technology:
    for h in EM.Hour:
        if EM.TagDispatchableTechnology[t] != 0:
            EM.CapacityFactor[t,h] = 1

#### Variables

In [201]:
#Define the variables here

#Creating the variables
EM.TotalCost = pyo.Var(EM.Technology, domain=pyo.NonNegativeReals)
EM.FuelProductionByTechnology = pyo.Var(EM.Hour, EM.Technology, EM.Fuel, domain=pyo.NonNegativeReals)
EM.Capacity = pyo.Var(EM.Technology, domain=pyo.NonNegativeReals)
EM.FuelUseByTechnology = pyo.Var(EM.Hour, EM.Technology, EM.Fuel, domain=pyo.NonNegativeReals)
EM.TechnologyEmissions = pyo.Var(EM.Technology)
EM.Curtailment = pyo.Var(EM.Hour,EM.Fuel, domain=pyo.NonNegativeReals)
EM.StorageEnergyCapacity = pyo.Var(EM.Storage,EM.Fuel, domain=pyo.NonNegativeReals) 
EM.StorageCharge = pyo.Var(EM.Hour,EM.Storage,EM.Fuel, domain=pyo.NonNegativeReals) 
EM.StorageDisCharge = pyo.Var(EM.Hour,EM.Storage,EM.Fuel, domain=pyo.NonNegativeReals) 
EM.StorageLevel = pyo.Var(EM.Hour,EM.Storage,EM.Fuel, domain=pyo.NonNegativeReals) 
EM.TotalStorageCost = pyo.Var(EM.Storage, domain=pyo.NonNegativeReals)


#### Objective Function

In [202]:
#Define the objective function here
#The objective is to minimize the total costs of the system

def obj_function(EM): 
    return pyo.summation(EM.TotalCost) +pyo.summation(EM.TotalStorageCost)

# create the objective object with the earlier created objective function
EM.obj = pyo.Objective(rule=obj_function)


#### Constraints

In [203]:
#Define the constraint functions/expressions and create constraint objects

#Writing the constraint functions
def demand_adequacy(EM,h,f): 
    return (
        sum(EM.FuelProductionByTechnology[h,t,f] for t in EM.Technology) 
        + sum(EM.StorageDisCharge[h,s,f] for s in EM.Storage if EM.StorageDisChargeEfficiency[s,f]>0) == 
        EM.Demand[f]*EM.DemandProfile[f,h] 
        + sum(EM.FuelUseByTechnology[h,t,f] for t in EM.Technology) 
        + EM.Curtailment[h,f]
        + sum(EM.StorageCharge[h,s,f] for s in EM.Storage if EM.StorageDisChargeEfficiency[s,f]>0) 
    )
        
def productioncost(EM,t): 
    return sum(EM.FuelProductionByTechnology[h,t,f] for f in EM.Fuel for h in EM.Hour) * EM.VariableCost[t] + EM.Capacity[t] * EM.CapitalCost[t] == EM.TotalCost[t]

def productionfunction_dispatchable(EM,h,t,f):
    if EM.TagDispatchableTechnology[t] > 0:
        return EM.OutputRatio[t,f] * EM.Capacity[t] * EM.CapacityFactor[t,h] >= EM.FuelProductionByTechnology[h,t,f]
    else:
        return pyo.Constraint.Skip

def productionfunction_variable(EM,h,t,f): 
    if EM.TagDispatchableTechnology[t] == 0:
        return EM.OutputRatio[t,f] * EM.Capacity[t] * EM.CapacityFactor[t,h] == EM.FuelProductionByTechnology[h,t,f]
    else:
        return pyo.Constraint.Skip
    
def usefunction(EM,h,t,f): 
    return EM.InputRatio[t,f] * sum(EM.FuelProductionByTechnology[h,t,ff] for ff in EM.Fuel) == EM.FuelUseByTechnology[h,t,f]

def technologyemissionfunction(EM,t):
    return sum(EM.FuelProductionByTechnology[h,t,f] for f in EM.Fuel for h in EM.Hour) * EM.EmissionRatio[t] == EM.TechnologyEmissions[t]

def emissionlimitconstraint(EM):
    return sum(EM.TechnologyEmissions[t] for t in EM.Technology) <= EM.EmissionLimit

def maxcapacityconstraint(EM,t):
    return EM.Capacity[t] + EM.ResidualCapacity[t] <= EM.MaxCapacity[t]

def storagechargefunction(EM,s,h,f): 
    if EM.StorageDisChargeEfficiency[s,f] > 0:
        return EM.StorageCharge[h,s,f] <= EM.StorageEnergyCapacity[s,f]/EM.E2PRatio[s]
    else:
        return pyo.Constraint.Skip

def storagedischargefunction(EM,s,h,f): 
    if EM.StorageDisChargeEfficiency[s,f] > 0:
        return EM.StorageDisCharge[h,s,f] <= EM.StorageEnergyCapacity[s,f]/EM.E2PRatio[s]
    else:
        return pyo.Constraint.Skip

def storagelevelfunction(EM,s,h,f): 
    if EM.StorageDisChargeEfficiency[s,f] > 0 and h>1:
        return EM.StorageLevel[h,s,f] == EM.StorageLevel[h-1,s,f] + EM.StorageCharge[h,s,f]*EM.StorageChargeEfficiency[s,f] - EM.StorageDisCharge[h,s,f]/EM.StorageDisChargeEfficiency[s,f]
    else:
        return pyo.Constraint.Skip

def storagelevelstartfunction(EM,s,h,f): 
    if EM.StorageDisChargeEfficiency[s,f] > 0 and h==1:
        return EM.StorageLevel[h,s,f] == 0.5*EM.StorageEnergyCapacity[s,f] + EM.StorageCharge[h,s,f]*EM.StorageChargeEfficiency[s,f] - EM.StorageDisCharge[h,s,f]/EM.StorageDisChargeEfficiency[s,f]
    else:
        return pyo.Constraint.Skip

def maxstoragelevelfunction(EM,s,h,f): 
    if EM.StorageDisChargeEfficiency[s,f] > 0:
        return EM.StorageLevel[h,s,f] <= EM.StorageEnergyCapacity[s,f]
    else:
        return pyo.Constraint.Skip
    
def storagecostfunction(EM,s): 
        return EM.TotalStorageCost[s] == sum(EM.StorageEnergyCapacity[s,f]*EM.InvestmentCostStorage[s] for f in EM.Fuel)


def storageannualbalancefunction(EM,s,f): 
    if EM.StorageDisChargeEfficiency[s,f] > 0:
        return sum(EM.StorageCharge[h,s,f] for h in EM.Hour)*EM.StorageChargeEfficiency[s,f] - sum(EM.StorageDisCharge[h,s,f] for h in EM.Hour) / EM.StorageDisChargeEfficiency[s,f] == 0
    else:
        return pyo.Constraint.Skip
    
#Creating the constraint objects
EM.DemandAdequacy = pyo.Constraint(EM.Hour,EM.Fuel, rule=demand_adequacy)
EM.ProductionCost = pyo.Constraint(EM.Technology, rule=productioncost)
EM.ProductionFunction_disp = pyo.Constraint(EM.Hour,EM.Technology, EM.Fuel, rule=productionfunction_dispatchable)
EM.ProductionFunction_res = pyo.Constraint(EM.Hour,EM.Technology, EM.Fuel, rule=productionfunction_variable)
EM.UseFunction = pyo.Constraint(EM.Hour,EM.Technology, EM.Fuel, rule=usefunction)
EM.TechnologyEmissionFunction = pyo.Constraint(EM.Technology, rule=technologyemissionfunction)
EM.EmissionLimitConstraint = pyo.Constraint(rule=emissionlimitconstraint)
EM.MaxCapacityConstraint = pyo.Constraint(EM.Technology, rule=maxcapacityconstraint)
EM.StorageChargeFunction = pyo.Constraint(EM.Storage,EM.Hour,EM.Fuel, rule=storagechargefunction)
EM.StorageDisChargeFunction = pyo.Constraint(EM.Storage,EM.Hour,EM.Fuel, rule=storagedischargefunction)
EM.StorageLevelFunction = pyo.Constraint(EM.Storage,EM.Hour,EM.Fuel, rule=storagelevelfunction)
EM.StorageLevelStartFunction = pyo.Constraint(EM.Storage,EM.Hour,EM.Fuel, rule=storagelevelstartfunction)
EM.MaxStorageLevelFunction = pyo.Constraint(EM.Storage,EM.Hour,EM.Fuel, rule=maxstoragelevelfunction)
EM.StorageCostFunction = pyo.Constraint(EM.Storage, rule=storagecostfunction)
EM.StorageAnnualBalanceFunction = pyo.Constraint(EM.Storage,EM.Fuel, rule=storageannualbalancefunction)

#### Model Instance

In [204]:
#Create the instance of the model here
instance = EM.create_instance()


#### Solver

In [205]:
#Define the solver here
opt = pyo.SolverFactory('gurobi')

#Solve the model through the instance of the model
opt.solve(instance)

{'Problem': [{'Name': 'x5783', 'Lower bound': 4.5596061273059325, 'Upper bound': 4.5596061273059325, 'Number of objectives': 1, 'Number of constraints': 5182, 'Number of variables': 5063, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 5063, 'Number of nonzeros': 13877, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.015000104904174805', 'Error rc': 0, 'Time': 0.19750761985778809}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

### Solution and Results

In [206]:
#Display results
instance.display()

Model unknown

  Variables:
    TotalCost : Size=6, Index=Technology
        Key             : Lower : Value               : Upper : Fixed : Stale : Domain
        DieselGenerator :     0 :                 0.0 :  None : False : False : NonNegativeReals
          DieselImports :     0 :                 0.0 :  None : False : False : NonNegativeReals
         ElectricHeater :     0 : 0.44777399999999984 :  None : False : False : NonNegativeReals
             HydroPower :     0 :                 0.0 :  None : False : False : NonNegativeReals
                SolarPV :     0 : 0.06945331321497122 :  None : False : False : NonNegativeReals
            WindOnshore :     0 :  2.4640156027529123 :  None : False : False : NonNegativeReals
    FuelProductionByTechnology : Size=2160, Index=FuelProductionByTechnology_index
        Key                                : Lower : Value                 : Upper : Fixed : Stale : Domain
          (1, 'DieselGenerator', 'Diesel') :     0 :                   

#### Export Results

In [207]:
#Extract the data from the solution to export results

#we need the demand profiles of all fuels, even those where we did not give a default value
demandprofile = pd.Series(instance.DemandProfile.extract_values(),name='value',index=pd.MultiIndex.from_tuples(instance.DemandProfile.extract_values().keys(),names=['fuel', 'hour']))

# generate data frames out of our results
ProductionByTechnology = pd.DataFrame(instance.FuelProductionByTechnology.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.FuelProductionByTechnology.extract_values().keys(), names=('Hour','Technology', 'Fuel')), columns=['Value']).reset_index()
Capacity = pd.DataFrame(instance.Capacity.extract_values().values(), index=(instance.Capacity.extract_values().keys()), columns=['Value']).rename_axis('Technology').reset_index()
Demand = pd.DataFrame(readin_demand * demandprofile *(-1), columns=['value']).reset_index().rename(columns={'fuel': 'Fuel', 'hour': 'Hour', 'value' : 'Value'})
Use = pd.DataFrame(instance.FuelUseByTechnology.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.FuelUseByTechnology.extract_values().keys(), names=('Hour','Technology', 'Fuel')), columns=['Value']).reset_index()
Curtailment = pd.DataFrame(instance.Curtailment.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.Curtailment.extract_values().keys(), names=('Hour','Fuel')), columns=['Value']).reset_index()
StorageLevel = pd.DataFrame(instance.StorageLevel.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.StorageLevel.extract_values().keys(), names=('Hour','Storage','Fuel')), columns=['Value']).reset_index().fillna(0)

#let's add outputs for our storage flows
StorageCharge = pd.DataFrame(instance.StorageCharge.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.StorageCharge.extract_values().keys(), names=('Hour','Technology','Fuel')), columns=['Value']).reset_index().fillna(0)
StorageDisCharge = pd.DataFrame(instance.StorageDisCharge.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.StorageDisCharge.extract_values().keys(), names=('Hour','Technology','Fuel')), columns=['Value']).reset_index().fillna(0)

# make sure that the values for Curtailment and Use are negative for our outputs
Use['Value']= Use['Value']*-1
Curtailment['Value']= Curtailment['Value']*-1
StorageCharge['Value']= StorageCharge['Value']*-1

# combine all values for Production, Use, Demand, and Curtailment into one data frame
EnergyBalance = pd.concat([ProductionByTechnology,Use], ignore_index=True)
EnergyBalance = pd.concat([EnergyBalance,Demand], ignore_index=True)
EnergyBalance['Technology'] = EnergyBalance['Technology'].fillna('Demand')
EnergyBalance = pd.concat([EnergyBalance,Curtailment], ignore_index=True)
EnergyBalance['Technology'] = EnergyBalance['Technology'].fillna('Curtailment')
EnergyBalance = pd.concat([EnergyBalance,StorageCharge], ignore_index=True)
EnergyBalance = pd.concat([EnergyBalance,StorageDisCharge], ignore_index=True)

# write our output values to csv files in the output subfolder
EnergyBalance.to_csv('../3. Output/EnergyBalance.csv',index=False)
Capacity.to_csv('../3. Output/Capacity.csv',index=False)
StorageLevel.to_csv('../3. Output/StorageLevel.csv',index=False)

### 2. Future Scenarios

###### (Define the modelled scenario here)

In [208]:
#Enter your code here